In [ ]:
import numpy as np
import pandas as pd

In [1]:
r_band, g_band, b_band = None, None, None
affine_transform, src_to_band_transformer, band_to_src_transformer = None, None, None

In [2]:
from pyproj import CRS, Transformer
import rasterio

INPUT_CRS = CRS.from_epsg(4326)  # WGS84

def setup_geospatial_data(rgb_tif_path):
    """
    1. Abre el TIFF multibanda.
    2. Lee R, G y B en arrays separados.
    3. Lee la transformada afín y el CRS del dataset.
    4. Crea transformers WGS84 <-> CRS del TIFF.
    """
    global r_band, g_band, b_band
    global affine_transform, src_to_band_transformer, band_to_src_transformer

    with rasterio.open(rgb_tif_path) as ds:
        # Banda 1 → Rojo, 2 → Verde, 3 → Azul
        r_band = ds.read(1)
        g_band = ds.read(2)
        b_band = ds.read(3)

        affine_transform = ds.transform
        band_crs = ds.crs

        # Transformadores coordenadas
        src_to_band_transformer = Transformer.from_crs(INPUT_CRS, band_crs, always_xy=True)
        band_to_src_transformer = Transformer.from_crs(band_crs, INPUT_CRS, always_xy=True)


In [3]:
import datetime
rgb_tif_path = "data/IN/Lalo Campos_Abril.tif"

start = datetime.datetime.now()
print(start, "Configurando datos geoespaciales RGB")
setup_geospatial_data(rgb_tif_path)
print("¡Listo! Tiempo:", datetime.datetime.now() - start)


2025-06-15 19:45:59.945016 Configurando datos geoespaciales RGB
¡Listo! Tiempo: 0:00:46.275503


In [4]:
from rasterio.transform import rowcol

def get_reflectance_rgb(lat, lon):
    # Pasa lon/lat → coordenadas del TIFF
    x, y = src_to_band_transformer.transform(lon, lat)
    row, col = rowcol(affine_transform, x, y)

    # Comprueba límites
    if not (0 <= row < r_band.shape[0] and 0 <= col < r_band.shape[1]):
        return None

    # Devuelve R, G, B
    return (
      int(r_band[row, col]),
      int(g_band[row, col]),
      int(b_band[row, col])
    )


In [ ]:
def compute_exg(r, g, b):
    return 2*g - r - b

# Ejemplo de máscara para todo el raster:
exg = compute_exg(r_band, g_band, b_band)
veg_mask = exg > 20   # ajusta el umbral según tus datos

# Para un DataFrame vectorizado:
rows, cols = np.indices(r_band.shape)
df_rgb = pd.DataFrame({
    'row': rows.ravel(), 'col': cols.ravel(),
    'R': r_band.ravel(), 'G': g_band.ravel(), 'B': b_band.ravel(),
    'ExG': exg.ravel()
})
df_rgb = df_rgb[df_rgb.ExG > 20]
